In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
from importlib import reload
import MobileNet_v3
import functions
MobileNet_v3 = reload(MobileNet_v3)

functions = reload(functions)
from functions import train, accuracy, MyDataLoader
from MobileNet_v3 import get_model as gm3


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.cuda.empty_cache()

In [ ]:
mobilenet = gm3().to(DEVICE)

In [ ]:
# optimizer settings
learning_rate = 1e-4

# train settings
epochs_count = 20

# dataset settings
batch_size = 32

optimizer = torch.optim.RMSprop(mobilenet.parameters(),
                                lr=learning_rate,
                                momentum=0.9)
loss_func = nn.NLLLoss()

train_transformer = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([.25], [.3])
])

mnist_train = datasets.MNIST('data/',
                             transform=train_transformer,
                             download=True)
mnist_test = datasets.MNIST('data/', train=False)

train_loader = MyDataLoader(mnist_train, batch_size, shuffle=True)
test_loader = MyDataLoader(mnist_test, batch_size, shuffle=False)

In [ ]:
train_history, best_parameters = \
    train(mobilenet, train_loader, loss_func, optimizer, epochs_count, accuracy)